Dear teachers, i was doing the lab 04 assignment and successfully find itemsets of size 2 with count = 8831 (itemsets.txt and support = 100). I was
having problems with memory when working with size 3. Then I tried with bigger support value = 300 to see if it works better but the problem's still there. My slave
machines died bc of full disk and I have to delete and re-setup all. I also put some works of finding confidences below please come to check. Thank you!

In [1]:
from pyspark import SparkContext, SparkConf
import re

In [2]:
import itertools
from functools import reduce

In [3]:
import math

In [4]:
conf = SparkConf().setMaster("yarn").setAppName("Lab04")

In [5]:
sc = SparkContext.getOrCreate (conf = conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [39]:
isDebug = True
support = 300

In [7]:
if not isDebug:
    sc.getConf().getAll()

Some pre-processsing steps. From initial RDD, I map each line to a list of integers

In [40]:
lineRDD = (sc.textFile("hdfs://master:9000/data/itemsets.txt", 15))

In [41]:
lineRDD = lineRDD.map(lambda line: line.strip().split(' '))

In [42]:
def convert_to_integer(line):
    line = list(set(line))
    new_line = []
    for i in range(len(line)):
        new_line.append(int(line[i]))
    return new_line

In [43]:
lineRDD = lineRDD.map(lambda line: convert_to_integer(line))

In [45]:
countLineRDD = lineRDD.flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda a , b: a + b).filter(lambda x: x[1] >= support )

In [46]:
supportRdd = countLineRDD

In [15]:
'''
Flatten a list of list and integer
'''
def flatten_list(tuple_line):
    new_list_line = list(tuple_line[0])
    new_list_line.append(tuple_line[1])
    return tuple(new_list_line)

In [16]:
'''
Flatten a list of list
'''
def flatten_list_of_list(tuple_line):
    new_list_line = list(tuple_line[0])
    for i in tuple_line[1]:
        new_list_line.append(i)
    return tuple(new_list_line)

In [17]:
'''
Compute corpus support list for each line in RDD (each line represents for a list of integers.
In order to make it, I use itertools.product function with some additional logics to 
find correct corpus set
For example:
[1,2,3,4] phase 2 -> [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)]
[1,2,3,4] phase 3 -> [(1,2,3), (1,2,4), (2,3,4), (1,3,4)]

  - To reuse data, corpus of phase n + 1 can be calculated from corpus of phase n
 '''
def computeCorpusSupportList(previous_list, base_list , phase):
    cartesian_list = list(itertools.product(previous_list, base_list))
    if phase > 2:
        cartesian_list = [flatten_list(item) for item in cartesian_list]
    cartesian_list = [(tuple(sorted(set(item))), 1) for item in cartesian_list] 
    cartesian_hash = {}
    for (a, b) in cartesian_list:
        if cartesian_hash.get(a) == None:
            cartesian_hash[a] = 0 
            
        cartesian_hash[a] += b
    
    cartesian_list = [ key for key in cartesian_hash if cartesian_hash[key] >= phase and len(key) == phase]
    return cartesian_list

In [18]:
'''
Find candidate support list. After each phase of counting support list, we want to propose the next support
list based on previous one
To make it, I use .cartesian() function of RDD that is similar to itertools.product() but used for RDD
'''
def computeCandidateSupportRDD(rdd, phase):
    cartesianRdd = rdd.map(lambda x: x[0])
    cartesianRdd = cartesianRdd.cartesian(cartesianRdd)
    if phase > 2 :
        cartesianRdd = cartesianRdd.map(lambda x: flatten_list_of_list(x))
    cartesianRdd = cartesianRdd.map(lambda x: (tuple(sorted(set(x))), 1))
    cartesianRdd = cartesianRdd.reduceByKey(lambda a , b: a+b)
    cartesianRdd = cartesianRdd.filter(lambda x: x[1] >= math.factorial(phase) and len(x[0]) == phase)
    cartesianRdd = cartesianRdd.map(lambda x: (x[0], 1))
    return cartesianRdd

In [47]:
current_phase = 1
corpus_lineRdd = lineRDD.map(lambda line: (line, line))
union_rdd = supportRdd

In [20]:
# current_phase = current_phase + 1
    
# print("current phase = ", current_phase)
    
#     # find candidate from previous supportRdd 
# candidate_supportRdd = computeCandidateSupportRDD(supportRdd, current_phase)

In [21]:
# candidate_supportRdd.take(3)

In [22]:
# corpus_lineRdd = corpus_lineRdd.map(lambda line: (computeCorpusSupportList(line[0] , line[1], current_phase) , line[1])) 

In [23]:
# # apply mapReduce for the corpus
# corpus_count_rdd = corpus_lineRdd.flatMap(lambda x: x[0]).map(lambda x: (x, 1))
# corpus_count_rdd = corpus_count_rdd.reduceByKey(lambda a, b : a + b).filter(lambda z : z[1] >= support)
        
# # find intersection
# supportRdd = candidate_supportRdd.join(corpus_count_rdd).map(lambda x: (x[0], x[1][1]))
    
# union_rdd.union(supportRdd)
# print(supportRdd.count())

In [48]:
'''
Find frequent itemsets of size 2.
The idea is similar to standard A_prior but instead of check if each item in candidate sets is available in every order and
then apply map-reduce method, I apply map-reduce for corpus first, and then find intersection later. 
'''
while True:
    
    print("current phase = ", 2)
    
    # find candidate from previous supportRdd 
    candidate_supportRdd = computeCandidateSupportRDD(supportRdd, 2)
    
    print("Done candidate_supportRdd")
    
    # find corpus for each line. through out the time, corpus_lineRdd is updated but still keep base "line[1]"
    corpus_lineRdd = corpus_lineRdd.map(lambda line: (computeCorpusSupportList(line[0] , line[1], 2) , line[1]))    
    
    print("Done corpus_lineRdd")
    
    # apply mapReduce for the corpus
    corpus_count_rdd = corpus_lineRdd.flatMap(lambda x: x[0]).map(lambda x: (x, 1))
    corpus_count_rdd = corpus_count_rdd.reduceByKey(lambda a, b : a + b).filter(lambda z : z[1] >= support)
        
    # inner join 2 rdd to get intersections 
    supportRdd = candidate_supportRdd.join(corpus_count_rdd).map(lambda x: (x[0], x[1][1]))
    
    union_rdd = union_rdd.union(supportRdd)
    print(supportRdd.count())
    break

current phase =  2
Done candidate_supportRdd
Done corpus_lineRdd


1816


In [25]:
'''
Number of requent itemsets of size 2 which has minSupport 100 is 8831
Number of requent itemsets of size 2 which has minSupport 300 is 1816

To find all frequent itemsets, just run the while loop until supportRdd is empty. Due to large computation,
I would like to find sets of size 2 and 3. 
'''

'\nNumber of requent itemsets of size 2 which has minSupport 100 is 8831\n'

In [26]:
# A = sc.parallelize([(1,10), (2,20), (3, 30), (4,40)])
# B = sc.parallelize([(9, 1), (2, 1), (3, 1)])
# B.join(A).collect()

In [ ]:
'''
For size = 3, some values can be reused to save computing resources. 
'''
while True:    
    print("current phase = ", 3)
    
    # find candidate from previous supportRdd 
    candidate_supportRdd = computeCandidateSupportRDD(supportRdd, 3)
    
    print("Done candidate_supportRdd")
    
    # find corpus for each line. through out the time, corpus_lineRdd is updated but still keep base "line[1]"
    corpus_lineRdd = corpus_lineRdd.map(lambda line: (computeCorpusSupportList(line[0] , line[1], 3) , line[1]))    
    
    print("Done corpus_lineRdd")
    
    # apply mapReduce for the corpus
    corpus_count_rdd = corpus_lineRdd.flatMap(lambda x: x[0]).map(lambda x: (x, 1))
    corpus_count_rdd = corpus_count_rdd.reduceByKey(lambda a, b : a + b).filter(lambda z : z[1] >= support)
        
    # find intersection
    supportRdd = candidate_supportRdd.join(corpus_count_rdd).map(lambda x: (x[0], x[1][1]))
    
    union_rdd = union_rdd.union(supportRdd)
    print(supportRdd.count())
    break

current phase =  3
Done candidate_supportRdd
Done corpus_lineRdd


22/12/09 09:57:51 WARN HeartbeatReceiver: Removing executor 2 with no recent heartbeats: 160309 ms exceeds timeout 120000 ms
22/12/09 09:57:51 WARN HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 134399 ms exceeds timeout 120000 ms


22/12/09 09:59:52 ERROR Utils: Uncaught exception in thread kill-executor-thread
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [120 seconds]. This timeout is controlled by spark.rpc.askTimeout
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.scheduler.cluster.CoarseGrainedSchedulerBackend.killExecutors(CoarseGrainedSchedulerBackend.scala:890)
	at org.apache.spark.SparkContext.killAndReplaceExecutor(SparkContext.scala:1825)
	at org.apache.spark.HeartbeatReceiver$$anon$2.$anonfun$run$2(HeartbeatReceiver.scala:214)
	at org.apache.spark.ut

In [28]:
'''
Number of frequent itemsets of size 3 which has minSupport 100 is
Number of frequent itemsets of size 3 which has minSupport 300 is 
'''

'\nNumber of requent itemsets of size 3 which has minSupport 100 is \n'

In [29]:
'''
To use result of problem 1 for problem 2, each time when finding out the new support sets, I unify it to union_rdd
in the format (A, k) with A is itemset (can be single value or a list), k is support of A in dataset
'''
union_rdd.count()

2346

In [ ]:
'''
Filter out itemsets of size 2 and 3.
'''
freq_item_set_tripleton = union_rdd.filter(lambda x: isinstance(x[0], tuple) and len(x[0]) == 3)
freq_item_set_doubleton = union_rdd.filter(lambda x: isinstance(x[0], tuple) and len(x[0]) == 2)

In [ ]:
freq_item_set_tripleton.take(10)

In [ ]:
freq_item_set_doubleton.take(10)

In [36]:
'''
The idea is that, to find confidence for each association rules (X,Y) -> Z, we have to compute support for (X,Y)
which is available in doubleton sets. Because of that, I want to find a way to make (X,Y) as keys and use join() operation.
'''
rdd1 = freq_item_set_tripleton.flatMap(lambda line: [(((line[0][0], line[0][1]), line[0][2]), line[1]), (((line[0][1], line[0][2]), line[0][0]), line[1]), (((line[0][0], line[0][2]), line[0][1]) ,line[1])]).map(lambda x: x)

In [ ]:
rdd1 = rdd1.map(lambda x: (x[0][0] , x))
rdd_join = rdd1.join(freq_item_set_doubleton) # Now we can find confidence values by some divisions